In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os, shutil, glob, cv2, tqdm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.backend as K
import pickle


비디오 to 이미지
=============

In [ ]:
vidcap = cv2.VideoCapture('/content/drive/MyDrive/multimedia/dance.mp4')
success,image = vidcap.read()

count = 1
success = True

while 1:
  success,image = vidcap.read()
  if (success):
    print(image.shape)
  else:
    break
  #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, (960,640))
  #image = cv2.add(image, (-30, -30, -30,0))
  #image[image<0] = 0
  
  cv2.imwrite("/content/drive/MyDrive/multimedia/originalimg/%d.png" % count, image)
  print("saved image %d.png" % count)
  
  if cv2.waitKey(10) == 27:                    
      break
  count += 1

In [ ]:

from skimage.metrics import structural_similarity
import argparse
import imutils
import cv2
 
# construct the argument parse and parse the arguments

# load the two input images
for i in range(619):
    imageA = cv2.imread(f'/content/drive/MyDrive/newstudy/mulimedia/o/{i+1}.png')
    imageB = cv2.imread('/content/drive/MyDrive/newstudy/mulimedia/backo.png')
    black = cv2.imread('/content/drive/MyDrive/newstudy/mulimedia/black.png')
    # if needed resize images using cv2.resize()
    # cv2.imshow("Original", imageA)
    # cv2.imshow("Modified", imageB)
    # cv2.waitKey(0)
    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
    (score, diff) = structural_similarity(grayA, grayB, full=True)
    diff = (diff * 255).astype("uint8")
    print(f"SSIM: {score}")
    thresh = cv2.threshold(
                 diff, 0, 255, 
                 cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU
             )[1]
    cnts, _ = cv2.findContours(
                thresh, 
                cv2.RETR_EXTERNAL, 
                cv2.CHAIN_APPROX_SIMPLE
              )
    for c in cnts:
        area = cv2.contourArea(c)
        if area > 500:
            x, y, w, h = cv2.boundingRect(c)
            #cv2.rectangle(imageA, (x, y), (x + w, y + h), (0, 0, 255), 2)
            filleda = imageA.copy()
            cv2.drawContours(black, [c],-1, (255, 255, 255), -1)
            cv2.drawContours(imageA, [c],-1, (255, 255, 255), 2)
            cv2.drawContours(filleda, [c],-1, (255, 255, 255), -1)
    cv2.imwrite(f"/content/drive/MyDrive/newstudy/mulimedia/black/1/{i+1}.png", imageA)
    cv2.imwrite(f"/content/drive/MyDrive/newstudy/mulimedia/black/2/{i+1}.png", black)
    cv2.imwrite(f"/content/drive/MyDrive/newstudy/mulimedia/black/3/{i+1}.png", black)
    display([imageA,black,filleda])

마스크 이미지 추출
====

In [ ]:
#배경 이미지 확인
mask = cv2.imread('/content/drive/MyDrive/newstudy/mulimedia/backo.png')
imgSlice = mask[:,:,:]
plt.imshow(imgSlice)
plt.show()

In [ ]:
#함수

def saveSlice(img, fname, path):
    img = np.uint8(img*255)
    fout = os.path.join(path, f'{fname}')
    cv2.imwrite(fout, img)
    print(f'[+] Slice saved: {fout}', end='\r')


def normalizemask(img):
    img = cv2.inRange(img, np.array([226,226,226]), np.array([255,255,255])) 
    return (img/255)

def display(display_list, opt=0):
    plt.figure(figsize=(30,30))
      
    title = [['Input Image', 'True Mask','Mixed Mask','ss'],['Input Image', 'True Mask', 'Predicted Mask','Mixed Mask'],['Predicted Mask', 'Predicted Mask','Mixed Mask']]
      
    for i in range(len(display_list)):
      plt.subplot(1, len(display_list), i+1)
      plt.title(title[opt][i])
      plt.imshow(display_list[i])
    plt.show()


In [ ]:
#마스크 파일 흰 부분만 저장하기
cnt = 0
for index, filename in enumerate(sorted(glob.iglob('/content/drive/MyDrive/newstudy/mulimedia/originalmask/*.png'))):
    m = cv2.imread(filename)
    mask = normalizemask(m)
    print(filename,mask.shape,np.max(mask))
    saveSlice(mask[:,:], os.path.basename(filename), '/content/drive/MyDrive/newstudy/mulimedia/mask')
    display([mask])
    shutil.copy(f'/content/drive/MyDrive/newstudy/mulimedia/originalimg/{os.path.basename(filename)}',f'/content/drive/MyDrive/newstudy/mulimedia/img/{os.path.basename(filename)}')
    cnt+=1
print(str(cnt)+'개의 이미지를 마스킹 해냈습니다.')

함수
==================

In [ ]:

    

def read_image(path):
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  #x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
  x = cv2.resize(x, (IMAGE_WIDTH,IMAGE_HEIGHT))
  x = x / 255.0
  x = x.astype(np.float32)
  return x
  

def predict_img(model,img,get_last=False): 

  prediction = model.predict(np.expand_dims(img, axis=0),verbose=0)

  if type(model.output) == list:
    num_of_output = len(model.output)
    x = sum(prediction) / num_of_output
  else: 
    x = prediction

  return x


def DiceBCELoss(y_true, y_pred,smooth=1e-6):    
    
    y_true_f= K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)

    BCE =  tf.keras.losses.binary_crossentropy(y_true_f, y_pred_f)
    intersection = K.sum(y_true_f * y_pred_f, axis=-1)
    dice_loss = 1 - (2 * intersection + smooth) / (K.sum(y_true_f,-1) + K.sum(y_pred_f,-1) + smooth)
    Dice_BCE = BCE + dice_loss
    
    return Dice_BCE
    
def IoULoss(y_true, y_pred, smooth=1e-6):

    y_pret_f = K.flatten(y_pred)
    y_true_f = K.flatten(y_true)
    
    intersection = K.sum(y_true_f* y_pret_f)
    total = K.sum(y_true_f) + K.sum(y_pret_f)
    union = total - intersection
    
    IoU = (intersection + smooth) / (union + smooth)
    return 1 - IoU


def FocalLoss(y_true, y_pred, alpha=1.0, gamma=2.0):    
    
    y_pret_f = K.flatten(y_pred)
    y_true_f = K.flatten(y_true)
    
    BCE = K.binary_crossentropy(y_true_f, y_pret_f)
    BCE_EXP = K.exp(-BCE)
    focal_loss = K.mean(alpha * K.pow((1-BCE_EXP), gamma) * BCE)
    
    return focal_loss
  
def Focal_IoU(y_true, y_pred):
  return FocalLoss(y_true,y_pred) + IoULoss(y_true, y_pred)

losses = {'DiceBCELoss': DiceBCELoss,'Focal_IoU':Focal_IoU}

모델적용
=======

In [ ]:
model=tf.keras.models.load_model('/content/drive/MyDrive/newstudy/mulimedia/model/Best1.h5',custom_objects=losses)
IMAGE_HEIGHT = 960
IMAGE_WIDTH = 640
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)
original = sorted(glob.glob('/content/drive/MyDrive/newstudy/mulimedia/originalimg/*.png'))

In [ ]:

vidcap = cv2.VideoCapture('/content/drive/MyDrive/newstudy/mulimedia/20221201_105040_1.mp4')
mask = np.zeros((int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)), IMAGE_HEIGHT, IMAGE_WIDTH),dtype='float32')
success,image = vidcap.read()

count = 0
success = True

while True:
  success,image = vidcap.read()
  if success==False: break
  x = image
  x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
  x = cv2.resize(x, (IMAGE_WIDTH,IMAGE_HEIGHT))
  x = x / 255.0
  x = x.astype(np.float32)
  p = predict_img(model,x)[0]
  mask[count,:,:] =  p[:,:,0]
  print("saved image %d.png" % count)
  
  if cv2.waitKey(10) == 27:                    
      break
  count += 1
np.save('/content/drive/MyDrive/newstudy/mulimedia/masks.npy', mask)

In [ ]:
mask= np.load('/content/drive/MyDrive/newstudy/mulimedia/masks.npy')


이미지 합치기(마스크파일 사용)
============

In [ ]:

filter = 0.5
filtered_maks = mask
filtered_maks[filtered_maks>=filter] = 1.0
filtered_maks[filtered_maks<filter] = 0.0
filtered_maks = filtered_maks.astype(np.uint8)

In [ ]:

frame_array  = []
bg_image = read_image('/content/drive/MyDrive/newstudy/mulimedia/mu.jpg')*255
for i in range(618):
  if (1):
    print(i)
    reversed_mask = 1 - filtered_maks[i,:,:]
    image = read_image(f'/content/drive/MyDrive/newstudy/mulimedia/originalimg/{i+1}.png') *255 # 이미지파일 읽어들이기
    cloth = cv2.bitwise_and(image, image, mask=filtered_maks[i,:,:]) # 옷 추출
    bg = cv2.bitwise_and(bg_image, bg_image, mask=reversed_mask) # 옷제외한 부분 추출한
    result = cloth+bg
    '''
    savedir=f'/content/drive/MyDrive/newstudy/mulimedia/graphs/{i}/'
    if os.path.exists(savedir)==False:
      os.mkdir(savedir)
    cv2.imwrite(f'{savedir}result.png', result)
    '''
    height, width, layers = result.shape
    size = (width,height)
    frame_array.append(result)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
allframe=618
sec=20
for i in range(1,allframe//2+1):
  for j in range(i+1,2*i):
    n=1
    sum=0
    while(n<=allframe):
      if (n%i ==0 or n%j==0):
        sum+=1
      n+=1
    if (sum/sec<=5 and sum/sec>=5 ):
      print(i,j,sum , sum/sec) #나온장수 ,프레임수


9 17 100 5.0


In [ ]:
fps = 5
a=9
b=17
pathOut = f'/content/drive/MyDrive/newstudy/mulimedia/result{fps}{a}{b}.mp4'
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
  if (i%a==0 or i%b==0):
    out.write(np.uint8(frame_array[i]))
out.release()
#fps만 조절하면 속도 조절
#장수 조절하면 스탑모션처럼
#6과 5가 나음

In [ ]:
fps = 6

pathOut = '/content/drive/MyDrive/newstudy/mulimedia/result.mp4'
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
  if (i%5==0):
    out.write(np.uint8(frame_array[i]))
out.release()
#fps만 조절하면 속도 조절
#장수 조절하면 스탑모션처럼
#6과 5가 나음

In [ ]:
fps = 30

pathOut = '/content/drive/MyDrive/newstudy/mulimedia/1.mp4'
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(glob.glob('/content/drive/MyDrive/newstudy/mulimedia/1/*.png'))):
  result = read_image(f'/content/drive/MyDrive/newstudy/mulimedia/1/{i+1}.png') *255 
  out.write(np.uint8(result))
  print(i+1)
out.release()


In [ ]:
fps = 3
pathOut = '/content/drive/MyDrive/newstudy/mulimedia/2.mp4'
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(glob.glob('/content/drive/MyDrive/newstudy/mulimedia/2/*.png'))):
  result = read_image(f'/content/drive/MyDrive/newstudy/mulimedia/2/{i+1}.png') *255 # 이미지파일 읽어들이
  out.write(np.uint8(result))
  print(np.max(result))
out.release()

0.0
255.0
255.0
255.0
255.0
255.0
255.0
255.0
255.0
255.0
255.0
255.0
